In [1]:
import numpy as np 
import pandas as pd 
import requests 
import xlsxwriter
import math

In [3]:
stocks = pd.read_csv('sp500.csv')
#stocks

In [4]:
from secrets import IEX_CLOUD_API_TOKEN

In [5]:
symbol = 'AMZN'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
#type(data)
#print(data) # Response = 200 means success
#print(data.status_code == 200)

dict

In [6]:
print(data)

{'symbol': 'AMZN', 'companyName': 'Amazon.com Inc.', 'primaryExchange': ' S(ESAGKQCGRETLD )/ENNM LSLABTAAO', 'calculationPrice': 'iexlasttrade', 'open': None, 'openTime': None, 'openSource': 'faofiilc', 'close': None, 'closeTime': None, 'closeSource': 'afclioif', 'high': None, 'highTime': None, 'highSource': None, 'low': None, 'lowTime': None, 'lowSource': None, 'latestPrice': 3432.71, 'latestSource': 'IEX Last Trade', 'latestTime': 'April 21, 2021', 'latestUpdate': 1635295446703, 'latestVolume': None, 'iexRealtimePrice': 3364.01, 'iexRealtimeSize': 103, 'iexLastUpdated': 1651533997090, 'delayedPrice': None, 'delayedPriceTime': None, 'oddLotDelayedPrice': None, 'oddLotDelayedPriceTime': None, 'extendedPrice': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPriceTime': None, 'previousClose': 3412.95, 'previousVolume': 2668209, 'change': 28.33, 'changePercent': 0.00822, 'volume': None, 'iexMarketPercent': 0.04320471374697192, 'iexVolume': 95454, 'avgTotalVolume': 33

In [10]:
price = data['latestPrice']
#print(price)
market_cap = data['marketCap']
#print(market_cap/1000000000000)

1.730722009858


In [11]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
#final_dataframe = pd.DataFrame([[0,0,0,0]],columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [12]:
final_dataframe.append(
    pd.Series(
        [
            symbol,
            price,
            market_cap,
            'N/A'
        ],
    index=my_columns
    ),
    ignore_index=True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AMZN,3432.71,1730722009858,N/A


In [16]:
final_dataframe = pd.DataFrame(columns=my_columns)
for stock in stocks['Ticker'][:10]:
#for stock in stocks['Ticker']:
    #print(stock)
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
            [
                stock,
                data['latestPrice'],
                data['marketCap'],
                'N/A'
            ],
            index=my_columns),
        ignore_index=True
        )

In [14]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,138.41,41790405379,N/A
1,AAL,21.00,14067870007,N/A
2,AAP,202.84,13641883441,N/A
3,AAPL,136.26,2267581397002,N/A
4,ABBV,115.50,202061946162,N/A
5,ABC,122.11,25322222109,N/A
6,ABMD,353.92,16263411250,N/A
7,ABT,125.30,227360478462,N/A
8,ACN,294.41,194678226416,N/A
9,ADBE,536.55,248256235126,N/A
